In [2]:
from recommendations import critics

print critics['Lisa Rose']['Lady in the Water']
print critics['Toby']['Snakes on a Plane']

print critics['Toby']


2.5
4.5
{'Snakes on a Plane': 4.5, 'Superman Returns': 4.0, 'You, Me and Dupree': 1.0}


# 3. 유사사용자 찾기
- 사람들이 선호하는 정보를 수집했다면 사람들 간의 취향이 얼마나 비슷한지 결정하는 방법을 찾아야 함
- 각각의 사람을 다른 모든 사람들과 비교해서 유사도(Similarity score)를 계산한다.
- 유클리디안 거리점수(euclidean distance score)와 피어슨 상관점수(Pearson correlation score)가 있음

## 3.1 유클리디안 거리점수

In [3]:
from math import sqrt
print sqrt( pow(5-4,2) + pow(2-1,2) )

1.41421356237


- 위 식은 가까운 사람일 수록 값이 작음
- 가독성을 위해 가까운 사람(상관이 높은사람)의 점수를 높히기 위해 역수를 취해줌
- 아래의 식과 같이 사용

In [4]:
1/(1+sqrt( pow(5-4,2) + pow(2-1,2) ))

0.4142135623730951

- 이 함수는 0~1 사이의 값을 리턴한다.
- 두사람이 동일한 선호도를 가지는 경우 1이 된다.
- 유사도 계산용 함수 작성

In [5]:
from math import sqrt

#person 1과 person 2의 거리 기반 유사도 점수 리턴 ( 유클리디안 거리 점수 )
def sim_distance(prefs,person1,person2):
    #공통 항목 추출
    si =[]
    sum_of_squares = 0.0
    for item in prefs[person1]:
        if item in prefs[person2]:
            si.append(item)
            
    #공통 항목이 없는 경우 0 리턴
    if len(si)==0:
        return 0
    
    # 모든 차이 값의 제곱을 더함
    for item in si:
        sum_of_squares = sum_of_squares + pow(prefs[person1][item] - prefs[person2][item],2)
        
    return 1/(1+sqrt(sum_of_squares))

In [6]:
sim_distance(critics,'Lisa Rose','Gene Seymour')

0.29429805508554946

## 3.2 피어슨 상관점수
- 사람간의 유사 관심을 측정하는 더 좋은 방법
- 좀 더 공식이 복잡하지만, 잘 정규화되지 않은 데이터의 경우에도 훨씬 좋은 결과를 제공한다.
- 피어슨 상관 점수의 좋은 특징은 점수 부풀리기(grade inflation)
- 피어슨 상관점수는 한 평론가가 다른 평론가에 비해 더 높은 점수를 주었어도, 그들 간의 점수 차이가 일정하다면 두 사람은 좋은 상관도를 가질 수 도 있다.
- 반면, 유클리디안 거리 점수는 실제 비슷한 취향을 가진 사람들 간에도 다른 사람에 비해 더 가혹하게 평가한다면 결과적으로 두 평론가가 다르다고 나올 수 도 있다.
- 적용하려는 추천 시스템에 따라 이러한 특성은 유익할 수도 그렇지 않을 수 도 있다.

- 피어슨 상관점수는 
  1. 공통으로 평가한 영화를 찾는다.
  2. 모든 선호도의 합을 구한다.
  3. 평가한 점수들의 제곱합을 구한다.
  4. 각 점수들의 곱의 합을 구한다.

In [7]:
#person1과 person2에 대한 피어슨 상관계수를 리턴
def sim_pearson(prefs,p1,p2):
    si=[]
    sum1 = 0
    sum2 = 0
    sum1Sq = 0
    sum2Sq = 0
    pSum = 0
    
    for item in prefs[p1]:
        if item in prefs[p2]:
            si.append(item)
    
    #요소들의 개수를 구함
    n = len(si)
    
    #공통 요소가 없으면 0 리턴
    if n == 0:
        return 0
    
    #모든 선호도를 합함
    for item in si:
        sum1 = sum1 + prefs[p1][item]
        sum2 = sum2 + prefs[p2][item]
        
    #제곱의 합을 계산
    for item in si:
        sum1Sq = sum1Sq + pow(prefs[p1][item],2)
        sum2Sq = sum2Sq + pow(prefs[p2][item],2)
        
    #곱의 합을 계산
    for item in si:
        pSum = pSum + (prefs[p1][item] * prefs[p2][item])
    
    #피어슨 상관계수 계산
    num = pSum - (sum1*sum2/n)
    den = sqrt((sum1Sq-pow(sum1,2)/n) * (sum2Sq-pow(sum2,2)/n))
    
    if den == 0:
        return 0
               
    r = num/den
    
    return r

In [8]:
sim_pearson(critics,'Lisa Rose','Gene Seymour')

0.39605901719066977

- 위 두가지 방법 외에도 자카드 계수(Jaccard coefficient)또는 맨해튼 거리(manhattan distance)가 있다.
- 상황에(데이터에) 따라 여러가지 상관도를 적용시켜보고 최적의 결과를 나타내는 방법을 적용시키면 된다.

## 3.3 평론가 순위

In [9]:
## 선호도 딕셔너리에서 최적의 상대를 구함
## 결과 개수와 유사도 함수는 옵션사항

def topMatches(prefs,person,n=5,similarity = sim_pearson):
    scores = []
    for other in prefs:
        if other != person:
            scores.append([(similarity(prefs,person,other),other)])
    
    #최고점이 상단에 오도록 목록을 정렬
    scores.sort()
    scores.reverse()
    return scores[0:n]

In [10]:
topMatches(critics,'Toby',n=3)

[[(0.9912407071619299, 'Lisa Rose')],
 [(0.9244734516419049, 'Mick LaSalle')],
 [(0.8934051474415647, 'Claudia Puig')]]

# 4. 항목 추천
- 유사 사용자를 찾았으면 컨텐츠를 추천해 주어야 한다.
- 나와 가장 유사한 취향을 가진 사람을 찾아 그가 좋아하는 컨텐츠 중 내가 소비하지 않은 컨텐츠를 찾을 수 있다.
- 이러한 방법은 내가 좋아할 만한 컨텐츠 중 몇몇을 리뷰하지 않은 사람이 나타날 수 도 있다.
- 또한, 다른 사람에게 악평을 받은 컨텐츠를 좋게 평가한 사용자를 알려줄 수 도 있다.
- 이러한 문제를 해결하기 위해 사용자 순위를 결정하는 가중 점수를 만들어 컨텐츠 점수를 계산해야 한다.

In [11]:
#다른 사람과의 순위의 가중평균값을 이용해서 특정 사람에게 추천
def getRecommendations(prefs,person,similarity=sim_pearson):
    totals = {}
    simSums = {}
    rankings = []
    for other in prefs:
        #나와 나를 비교하지 말것
        if other == person: 
            continue
        
        ##유사도 계산
        sim = similarity(prefs,person,other)
       
        
        #0 이하 점수는 무시함
        if sim <=0:
            continue
        
        for item in prefs[other]:
            #내가 보지 못한 영화만 대상
            if item not in prefs[person] or prefs[person][item] == 0:
                totals.setdefault(item,0)
                # 나와의 유사도 * 다른사람의 내가 보지 못한 영화에 대한 점수 ++
                totals[item] += prefs[other][item] * sim
                # 내가 보지 못한 영화에 대한 유사도 ++
                simSums.setdefault(item,0)
                simSums[item] += sim
        
    # 정규화된 목록 생성
    for item,total in totals.items():
        rankings.append([total/simSums[item],item])
        
    rankings.sort()
    rankings.reverse()
    
    return rankings

In [12]:
getRecommendations(critics,'Toby')

[[3.3477895267131013, 'The Night Listener'],
 [2.8325499182641614, 'Lady in the Water'],
 [2.5309807037655645, 'Just My Luck']]

# 5. 제품매칭
 - 지금까지 유사한 사람을 찾고 유사한 사람에게 컨텐츠를 추천하는 방법을 알아보았다.
 - 이제는 한 컨텐츠와 다른 컨텐츠가 얼마나 비슷한지 알아본다.
 - 이 경우 특정 컨텐츠를 좋아하는 사람들이 좋아한 다른 것을 살펴보아 유사도를 구한다.
 - 방법은 같고 위 방법에서 사람 <-> 컨텐츠 하면 된다.

In [13]:
# 선호도 행과 열 변경
def transformPrefs(prefs):
    result = {}
    for person in prefs:
        for item in prefs[person]:
            result.setdefault(item,{})
            
            #컨텐츠와 사람을 바꿈
            result[item][person] = prefs[person][item]
    return result

In [15]:
movies = transformPrefs(critics)
topMatches(movies,'Superman Returns')

[[(0.6579516949597695, 'You, Me and Dupree')],
 [(0.4879500364742689, 'Lady in the Water')],
 [(0.11180339887498941, 'Snakes on a Plane')],
 [(-0.1798471947990544, 'The Night Listener')],
 [(-0.42289003161103106, 'Just My Luck')]]

- 이 예제에서는 음수값이 리턴됨
- 음수값의 의미는 슈퍼맨 리턴즈를 좋아한 사람이 음수의 영화를 싫어하는 경향이 있음을 의미
- 마지막으로 영화에 대해 평론가를 추천할 수 있다.
- 현재 영화를 보지않은 사람에 대하여 유사한 영화에 점수를 높게준 평론가 추천

In [16]:
getRecommendations(movies,'Just My Luck')

[[4.0, 'Michael Phillips'], [3.0, 'Jack Matthews']]